In [1]:
import numpy as np
import os
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
alexaData = pickle.load(open('drive/MyDrive/AlexaFilteredData.pk1' , 'rb'))
angelData = pickle.load(open('drive/MyDrive/AngelFilteredData.pk1' , 'rb'))
angelData2 = pickle.load(open('drive/MyDrive/AngelFilteredData2.pk1' , 'rb'))
ashaniData = pickle.load(open('drive/MyDrive/AshaniFilteredData.pk1' , 'rb'))

In [3]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

trainingData = [[],[],[],[],[],[],[],[],[]]
trainingDataSegmented = [[[]],[[]],[[]],[[]],[[]],[[]],[[]],[[]],[[]]]

num0 = 0
num1 = 0

for i in range(0,9):
  #trainingData[i] = list(alexaData[i]) + list(angelData[i]) + list(angelData2[i]) + list(ashaniData[i])
  trainingData[i] = list(alexaData[i]) + list(ashaniData[i])
  for j in range(0, len(trainingData[i])):
    if j // 250 >= len(trainingDataSegmented[i]):
      #print(num1, num0)
      trainingDataSegmented[i].append([])
      if i == 8:
        if num1 < num0:
          #print("appending 0")
          trainingDataSegmented[i][(j // 250)-1].append(0)
        else:
          #print("appending 1")
          trainingDataSegmented[i][(j // 250)-1].append(1)
        num0 = 0
        num1 = 0
    if i != 8:
      trainingDataSegmented[i][j // 250].append(trainingData[i][j])
    else:
      if trainingData[i][j] == 1:
        #print("hi")
        num1+=1
      elif trainingData[i][j] == 0:
        num0+=1
  trainingDataSegmented[i].pop()


x = np.array([np.array(trainingDataSegmented[0]), np.array(trainingDataSegmented[1]), np.array(trainingDataSegmented[2]), np.array(trainingDataSegmented[3]), np.array(trainingDataSegmented[4]), np.array(trainingDataSegmented[5]), np.array(trainingDataSegmented[6]), np.array(trainingDataSegmented[7])])
x = np.swapaxes(x, 0, 1)

y = np.array(trainingDataSegmented[8])
tmp = []
for i in range(y.shape[0]): # This loop turns the sample readings into seconds
  tmp.append(y[i][0])
y = np.array(tmp)

# Split dataset into training set and test set
# x sample
# y 0 or 1

# now we need to "flatten" the 3d array into a 2d one
xsamples, x1, x2 = x.shape
x = x.reshape((xsamples,x1*x2))

print(x.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=109) # 70% training and 30% test



(2768, 2000) (2768,)


array([ 0.02594768, -0.03460123, -0.12587811, ...,  0.03362431,
        0.02650186, -0.05313976])

In [31]:
# scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = RobustScaler()
# X_train_scale = scaler.fit_transform(X_train)
# X_test_scale = scaler.transform(X_test)

X_train_scale = X_train
X_test_scale = X_test

X_train_scale.mean(axis=0)
#X_train_scale.std(axis=0)

array([ 0.02594768, -0.03460123, -0.12587811, ...,  0.03362431,
        0.02650186, -0.05313976])

In [38]:
#Import svm model
from sklearn import svm
from sklearn import linear_model

#Create a svm Classifier
classifier = svm.SVC(kernel='rbf', C=2)
#classifier = svm.LinearSVC(dual=False)
#classifier = linear_model.SGDClassifier(loss='squared_epsilon_insensitive')

#Train the model using the training sets
classifier.fit(X_train_scale, y_train)

#Predict the response for test dataset
y_pred = classifier.predict(X_test_scale)

In [39]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7509025270758123


In [40]:
import pickle

pickle.dump(classifier, open('focusmodel.pk1', 'wb'))